In [1]:
#!pip uninstall tensorflow
!pip3 install --upgrade tensorflow
!pip install grpcio>=1.24.3 --upgrade
!pip install -U keras-tuner
!pip install tensorflow-addons
!pip install keras-rectified-adam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.20.0-py3-none-any.whl size=8254 sha256=076ebcb269567310fc97d5776c300f3d6741de3ac4ed70bbf562287fa27b3a44
  Stored in directory: /root/.cache/pip/wheels/24/fe/0f/dcf898210ceef6ae45cea51d3d27765f38bce2b4dc8e88245f
Successfully built keras-rectified-adam


In [16]:
# %TensorFlow 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# if device_name != '/device:CPU:0':
#   raise SystemError('CPU device not found')
# print('Found CPU at: {}'.format(device_name))

In [11]:
import os
os.environ['TF_KERAS'] = '1'
!echo $TF_KERAS

1


In [12]:
from keras_radam import RAdam

In [13]:
import librosa
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import IPython.display as ipd
import librosa.display
import scipy
import glob
import numpy as np
import math
import warnings
import pickle
from sklearn.utils import shuffle

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [14]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12690697960889648526
 xla_global_id: -1]

In [17]:
tf.random.set_seed(999)
np.random.seed(999)

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [19]:
!ls 'drive/My Drive/datasets/tf_records'

ls: cannot access 'drive/My Drive/datasets/tf_records': No such file or directory


In [20]:
train_tfrecords_filenames = glob.glob('drive/My Drive/datasets/tf_records/train_*')
np.random.shuffle(train_tfrecords_filenames)
train_tfrecords_filenames = list(train_tfrecords_filenames)
print(train_tfrecords_filenames)
val_tfrecords_filenames = glob.glob('drive/My Drive/datasets/tf_records/val_*')

[]


In [21]:
windowLength = 256
overlap      = round(0.25 * windowLength) # overlap of 75%
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8
print("windowLength:",windowLength)
print("overlap:",overlap)
print("ffTLength:",ffTLength)
print("inputFs:",inputFs)
print("fs:",fs)
print("numFeatures:",numFeatures)
print("numSegments:",numSegments)

windowLength: 256
overlap: 64
ffTLength: 256
inputFs: 48000.0
fs: 16000.0
numFeatures: 129
numSegments: 8


In [22]:
mozilla_basepath = 'drive/My Drive/datasets/en'
UrbanSound8K_basepath = 'drive/My Drive/datasets/UrbanSound8K'

@article{
  silva2019audiodenoising,
  title={Practical Deep Learning Audio Denoising},
  author={Silva, Thalles Santos},
  journal={https://sthalles.github.io},
  year={2019}
  url={https://sthalles.github.io/practical-deep-learning-audio-denoising/}
}